In [ ]:
import rospy
import tf2_ros
from ros_tensorflow_msgs.srv import *
from rail_segmentation.srv import *
from rail_manipulation_msgs.srv import *
from geometry_msgs.msg import TransformStamped, Pose
from manipulation_test.srv import *
import numpy as np
from ros_numpy import numpify, msgify
import tf
import sys
import moveit_commander
import moveit_msgs.msg
from moveit_msgs.msg import RobotState
from sensor_msgs.msg import JointState
from sensor_msgs import point_cloud2
import random
from moveit_msgs.msg import AttachedCollisionObject
from moveit_msgs.msg import CollisionObject
from shape_msgs.msg import SolidPrimitive

In [ ]:
rospy.init_node('pick_and_place_node')

In [ ]:
# find the table first
rospy.wait_for_service('table_searcher/search_table')
table_searcher = rospy.ServiceProxy('table_searcher/search_table', SearchTable)


In [ ]:
table_info = table_searcher()

In [ ]:
# detect obstacle on the table
rospy.wait_for_service('table_searcher/segment_objects')
object_searcher = rospy.ServiceProxy('table_searcher/segment_objects', SegmentObjects)

In [ ]:
detected_objects = object_searcher()

In [ ]:
print("detected objects' number: ", len(detected_objects.segmented_objects.objects))

In [ ]:
# get camera transform from tf tree
tfBuffer = tf2_ros.Buffer()
listener = tf2_ros.TransformListener(tfBuffer)

In [ ]:
try:
    camera_trans = tfBuffer.lookup_transform('base_link', 'head_camera_rgb_optical_frame', rospy.Time())
except (tf2_ros.LookupException, tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException):
    print("tf error")

In [ ]:
rospy.wait_for_service('grasp_predict')

In [ ]:
grasp_predictor = rospy.ServiceProxy('grasp_predict', Predict)
try:
    predicted_grasp_result = grasp_predictor(table_info.full_point_cloud, detected_objects.segmented_objects.objects[0].point_cloud, camera_trans)
except rospy.ServiceException as e:
    print("Service call failed: %s"%e)

In [ ]:
moveit_commander.roscpp_initialize(sys.argv)
move_group = moveit_commander.MoveGroupCommander("arm") 
scene = moveit_commander.PlanningSceneInterface()

In [ ]:
table_info = table_searcher()
# add table into the planning scene
table_pose = geometry_msgs.msg.PoseStamped()
table_pose.header.frame_id = "base_link"
table_pose.pose.orientation.x = table_info.orientation.x
table_pose.pose.orientation.y = table_info.orientation.y
table_pose.pose.orientation.z = table_info.orientation.z
table_pose.pose.orientation.w = table_info.orientation.w
table_pose.pose.position.x = table_info.center.x
table_pose.pose.position.y = table_info.center.y
table_pose.pose.position.z = table_info.center.z / 2
table_name = "table"
scene.add_box(table_name, table_pose, size=(table_info.width, table_info.depth, table_info.center.z))

In [ ]:
grasp_shift = np.array([[1,0,0,0.05],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
pre_grasp_shift = np.array([[1,0,0,-0.1],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
pick_shift = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0.05],[0,0,0,1]])

for i in range(len(predicted_grasp_result.predicted_grasp_poses)):
    move_group.set_start_state_to_current_state()
    
    # generate grasp pose
    grasp_pose = numpify(predicted_grasp_result.predicted_grasp_poses[i].pose).dot(grasp_shift)

    # calculate the pre-grasp pose
    pre_grasp_pose = grasp_pose.dot(pre_grasp_shift)
    
    # calculate the pick-up pose
    pick_up_pose = pick_shift.dot(grasp_pose)

    trans = tf.transformations.translation_from_matrix(pre_grasp_pose).tolist()
    quat = tf.transformations.quaternion_from_matrix(pre_grasp_pose).tolist()

    move_group.clear_pose_targets()
    move_group.set_pose_target(trans + quat)
    plan_result = move_group.plan()
    if plan_result[0]:
        joint_state = JointState()
        joint_state.header.stamp = rospy.Time.now()
        joint_state.name = plan_result[1].joint_trajectory.joint_names
        joint_state.position = plan_result[1].joint_trajectory.points[-1].positions
        moveit_robot_state = RobotState()
        moveit_robot_state.joint_state = joint_state
        move_group.set_start_state(moveit_robot_state)
        (approach_plan, fraction) = move_group.compute_cartesian_path([msgify(geometry_msgs.msg.Pose, grasp_pose)], 0.01, 0.0)
        # check whether you can approach the object
        if fraction < 0.9:
            continue
        
        joint_state.header.stamp = rospy.Time.now()
        joint_state.position = approach_plan.joint_trajectory.points[-1].positions
        moveit_robot_state.joint_state = joint_state
        move_group.set_start_state(moveit_robot_state)
        (pick_plan, fraction) = move_group.compute_cartesian_path([msgify(geometry_msgs.msg.Pose, pick_up_pose)], 0.01, 0.0)
        # check whether you can pick the object
        if fraction < 0.9:
            continue
            
        print "got a way to pick up the object"
        
        break

move_group.clear_pose_targets()
scene.clear()

In [ ]:
object_pose = Pose()
object_pose.position.x = detected_objects.segmented_objects.objects[0].center.x
object_pose.position.y = detected_objects.segmented_objects.objects[0].center.y
object_pose.position.z = detected_objects.segmented_objects.objects[0].center.z
object_pose.orientation.x = detected_objects.segmented_objects.objects[0].orientation.x
object_pose.orientation.y = detected_objects.segmented_objects.objects[0].orientation.y
object_pose.orientation.z = detected_objects.segmented_objects.objects[0].orientation.z
object_pose.orientation.w = detected_objects.segmented_objects.objects[0].orientation.w
object_pose_mat = numpify(object_pose)

# get the table top pose
table_top_pose = geometry_msgs.msg.PoseStamped()
table_top_pose.header.frame_id = "base_link"
table_top_pose.pose.orientation.x = table_info.orientation.x
table_top_pose.pose.orientation.y = table_info.orientation.y
table_top_pose.pose.orientation.z = table_info.orientation.z
table_top_pose.pose.orientation.w = table_info.orientation.w
table_top_pose.pose.position.x = table_info.center.x
table_top_pose.pose.position.y = table_info.center.y
table_top_pose.pose.position.z = table_info.center.z

# get the object in hand pose used later for placing
in_hand_pose = np.linalg.inv(grasp_pose).dot(object_pose_mat) # input of placing
# get the object pose should be on the table(we should consider the table rotation as well)
table_pose_mat = numpify(table_top_pose.pose)
object_pose_on_table = np.linalg.inv(table_pose_mat).dot(object_pose_mat)
object_pose_on_table[0][3] = 0.0
object_pose_on_table[1][3] = 0.0
print "================= return to the planner ===================="
print "object pose in hand"
print in_hand_pose
print "object pose on the table top"
print object_pose_on_table

In [ ]:
## need to place object
# search a position to place
# get the table info
table_info = table_searcher()
# add table into the planning scene
table_pose = geometry_msgs.msg.PoseStamped()
table_pose.header.frame_id = "base_link"
table_pose.pose.orientation.x = table_info.orientation.x
table_pose.pose.orientation.y = table_info.orientation.y
table_pose.pose.orientation.z = table_info.orientation.z
table_pose.pose.orientation.w = table_info.orientation.w
table_pose.pose.position.x = table_info.center.x
table_pose.pose.position.y = table_info.center.y
table_pose.pose.position.z = table_info.center.z / 2
table_name = "table"
scene.add_box(table_name, table_pose, size=(table_info.width, table_info.depth, table_info.center.z))
table_pose_mat = numpify(table_pose.pose)

# get the points of the table top
points = list(point_cloud2.read_points(table_info.point_cloud, field_names=("x", "y", "z"), skip_nans=True))

In [ ]:
def rotate_pose_z(pose, theta):
    # Create the rotation matrix
    rotation_matrix = np.array([
        [np.cos(theta), -np.sin(theta), 0, 0],
        [np.sin(theta), np.cos(theta), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    # Multiply the pose by the rotation matrix
    rotated_pose = np.dot(pose, rotation_matrix)

    return rotated_pose
def rotate_pose_z_random(pose):
    theta = np.random.uniform(0, 2*np.pi)  # Random angle between 0 and 2*pi
    return rotate_pose_z(pose, theta)

In [ ]:
attached_object = AttachedCollisionObject()
attached_object.link_name = "wrist_roll_link"

# Create a CollisionObject
collision_object = CollisionObject()
collision_object.id = "object"
collision_object.header.frame_id = "base_link"

# Create a SolidPrimitive box
box = SolidPrimitive()
box.type = box.BOX
box.dimensions = [detected_objects.segmented_objects.objects[0].width, detected_objects.segmented_objects.objects[0].depth, detected_objects.segmented_objects.objects[0].height]  # Size of the box

collision_object.primitives = [box]
collision_object.primitive_poses = [msgify(geometry_msgs.msg.Pose, numpify(move_group.get_current_pose().pose).dot(in_hand_pose))]

# Add the collision object into the AttachedCollisionObject message
attached_object.object = collision_object
attached_object.object.operation = attached_object.object.ADD
attached_object.touch_links = ["l_gripper_finger_link", "r_gripper_finger_link", "gripper_link"]

In [ ]:
for j in range(10):
    move_group.set_start_state_to_current_state()
    
    # randomly select a point on the table and consider it as the table origin.
    table_pose_mat[:3, 3] = random.choice(points)
    
    place_pose_on_table = table_pose_mat.dot(rotate_pose_z_random(object_pose_on_table))
    
    hand_pose_for_place = place_pose_on_table.dot(np.linalg.inv(in_hand_pose))
    
    hand_pose_for_pre_place = pick_shift.dot(hand_pose_for_place)
    
    hand_pose_for_release = hand_pose_for_place.dot(pre_grasp_shift)

    trans = tf.transformations.translation_from_matrix(hand_pose_for_pre_place).tolist()
    quat = tf.transformations.quaternion_from_matrix(hand_pose_for_pre_place).tolist()

    move_group.clear_pose_targets()
    # need to attach the object on the end-effector
    moveit_robot_state = move_group.get_current_state()
    moveit_robot_state.attached_collision_objects = [attached_object]
    
    move_group.set_start_state(moveit_robot_state)
    
    move_group.set_pose_target(trans + quat)
    plan_result = move_group.plan()
    if plan_result[0]:
        joint_state = JointState()
        joint_state.header.stamp = rospy.Time.now()
        joint_state.name = plan_result[1].joint_trajectory.joint_names
        joint_state.position = plan_result[1].joint_trajectory.points[-1].positions
        moveit_robot_state = RobotState()
        moveit_robot_state.joint_state = joint_state
        move_group.set_start_state(moveit_robot_state)
        (place_plan, fraction) = move_group.compute_cartesian_path([msgify(geometry_msgs.msg.Pose, hand_pose_for_place)], 0.01, 0.0)
        # check whether you can place the object
        if fraction < 0.9:
            continue
            
        joint_state.header.stamp = rospy.Time.now()
        joint_state.position = place_plan.joint_trajectory.points[-1].positions
        moveit_robot_state.joint_state = joint_state
        move_group.set_start_state(moveit_robot_state)
        (release_plan, fraction) = move_group.compute_cartesian_path([msgify(geometry_msgs.msg.Pose, hand_pose_for_release)], 0.01, 0.0)
        # check whether you can pick the object
        if fraction < 0.9:
            continue
        print "find a way to place the object"
        break

move_group.clear_pose_targets()
scene.clear()